In [ ]:
!pip install selenium

In [ ]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
from math import ceil
import time
from selenium import webdriver

Getting URL

In [ ]:
url = "https://www.meesho.com/ethnicwear-women/pl/3tq"

In [ ]:
page_content = r.get(url).content

## Parsing the HTML

In [ ]:
soup = BeautifulSoup(page_content, 'html.parser')
#print(soup.prettify())

In [ ]:
all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))
all_class

[('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'ihwYlH'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'eCJiSA'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'bGpzcx'),
 ('sc-dkrFOg', 'ProductList__GridCol-sc-8lnc8o-0', 'cokuZA', 'jGAjKy')]

In [ ]:
import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))





In [ ]:
all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df= pd.DataFrame(all_data)
df

,Product Name,Ratings,Current price,Previous price,No of Review
0,TRENDY VOGUISH WOMEN KURTA SETS,4.3,460,489,44136
1,AAKARSHA GRACEFUL WOMEN KURTA SETS,4.2,354,354,36
2,JIVIKA VOGUISH WOMEN KURTA SETS,3.6,295,346,135782
3,AAGYEYI ATTRACTIVE SAREES,3.8,244,279,45819
4,AISHANI GRACEFUL KURTIS,3.7,266,266,15214
5,YAGNIK FASHION WOMEN'S KURTA SETS,3.9,609,799,2932
6,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.7,577,799,829
7,MYRA REFINED WOMEN KURTA SETS,3.7,396,435,29859
8,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.7,614,799,4007
9,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.9,614,899,3805


In [ ]:
df["Discount %"] = round(100- (df["Current price"]/df['Previous price'])*100)

In [ ]:
df

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,TRENDY VOGUISH WOMEN KURTA SETS,4.3,460,489,44136,6.0
1,AAKARSHA GRACEFUL WOMEN KURTA SETS,4.2,354,354,36,0.0
2,JIVIKA VOGUISH WOMEN KURTA SETS,3.6,295,346,135782,15.0
3,AAGYEYI ATTRACTIVE SAREES,3.8,244,279,45819,13.0
4,AISHANI GRACEFUL KURTIS,3.7,266,266,15214,0.0
5,YAGNIK FASHION WOMEN'S KURTA SETS,3.9,609,799,2932,24.0
6,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.7,577,799,829,28.0
7,MYRA REFINED WOMEN KURTA SETS,3.7,396,435,29859,9.0
8,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.7,614,799,4007,23.0
9,YAGNIK FASHION WOMEN'S KURTA SETS PACK OF 2,3.9,614,899,3805,32.0


In [ ]:
url = "https://www.meesho.com/jewellery-sets-women/pl/3jy"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl= pd.DataFrame(all_data)
df_jewl["Discount %"] = round(100- (df_jewl["Current price"]/df_jewl['Previous price'])*100)
df_jewl

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,PRINCESS ELEGANT JEWELLERY SETS,4.0,213,222,12458,4.0
1,DIVA GLITTERING JEWELLERY SETS,4.0,179,197,101278,9.0
2,SHIMMERING BEAUTIFUL JEWELLERY SETS,4.1,203,208,8645,2.0
3,JEWELLERY SET,4.2,211,229,13901,8.0
4,PRINCESS ELEGANT JEWELLERY SETS,4.0,166,173,28866,4.0
5,SHIMMERING BEAUTIFUL JEWELLERY SETS,4.0,202,229,4163,12.0
6,ALLURE UNIQUE JEWELLERY SETS,4.3,126,156,5923,19.0
7,ALLURE ELEGANT JEWELLERY SETS,3.6,194,202,0,4.0
8,PRINCESS CHIC JEWELLERY SETS,4.1,185,240,2475,23.0
9,TWINKLING FANCY JEWELLERY SETS,3.9,125,134,3393,7.0


In [ ]:
url = "https://www.meesho.com/earrings-women/pl/3lc"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_1= pd.DataFrame(all_data)
df_jewl_1["Discount %"] = round(100- (df_jewl_1["Current price"]/df_jewl_1['Previous price'])*100)
df_jewl_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,TRENDY EARRINGS & STUDS,4.0,160,168,6215,5.0
1,TRENDY OTHERS (EARRINGS DEP),3.9,159,159,36,0.0
2,LATEST EARRINGS & STUDS,4.2,126,131,5196,4.0
3,GRACEFUL EARRINGS & STUDS,4.2,155,181,54286,14.0
4,UNIQUE EARRINGS & STUDS,4.2,141,160,5082,12.0
5,WONDERFUL EARRINGS & STUDS,,657,657,0,0.0
6,FASHIONABLE EARRINGS & STUDS,,558,558,0,0.0
7,GRACEFUL EARRINGS & STUDS,4.2,155,181,16768,14.0
8,STYLES EARRINGS & STUDS,4.1,202,211,0,4.0
9,CASUAL EARRINGS & STUDS,3.9,361,361,0,0.0


In [ ]:
url = "https://www.meesho.com/mangalsutras/pl/3j5"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_2= pd.DataFrame(all_data)
df_jewl_2["Discount %"] = round(100- (df_jewl_2["Current price"]/df_jewl_2['Previous price'])*100)
df_jewl_2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ELITE FUSION MANGALSUTRAS,4.0,100,100,2702,0.0
1,FEMININE FUSION JEWELLERY SETS,,978,978,0,0.0
2,ELITE BEAUTIFUL MANGALSUTRAS,3.9,189,189,9901,0.0
3,SHIMMERING CHUNKY MANGALSUTRAS,4.1,141,147,19318,4.0
4,DIVA GRACEFUL MANGALSUTRAS,3.9,97,104,5748,7.0
5,PRINCESS FANCY JEWELLERY SETS,,978,978,0,0.0
6,PRINCESS FANCY MANGALSUTRAS,4.0,90,988,1916,91.0
7,ELITE COLORFUL MANGALSUTRAS,4.1,186,186,7052,0.0
8,MANGALSUTRA UNDER 50 NLAPUSALA FEMININE CHUNKY...,4.0,104,111,4491,6.0
9,PRINCESS FANCY MANGALSUTRAS,4.0,154,160,2585,4.0


In [ ]:
url = "https://www.meesho.com/mangalsutras/pl/3j5"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_2= pd.DataFrame(all_data)
df_jewl_2["Discount %"] = round(100- (df_jewl_2["Current price"]/df_jewl_2['Previous price'])*100)
df_jewl_2

In [ ]:
url = "https://www.meesho.com/stud-earrings-women/pl/3n9"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_3= pd.DataFrame(all_data)
df_jewl_3["Discount %"] = round(100- (df_jewl_3["Current price"]/df_jewl_3['Previous price'])*100)
df_jewl_3

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FASHIONABLE EARRINGS & STUDS,4.0,167,174,9032,4.0
1,FASHIONABLE EARRINGS & STUDS,,558,558,0,0.0
2,LATEST EARRINGS & STUDS,4.2,126,131,5196,4.0
3,GRACEFUL EARRINGS & STUDS,4.2,155,181,54286,14.0
4,STYLES EARRINGS & STUDS,4.0,144,149,7456,3.0
5,CLASSIC EARRINGS & STUDS,4.0,123,141,745,13.0
6,STYLO EARRINGS & STUDS,4.2,191,191,0,0.0
7,CLASSIC EARRINGS & STUDS,4.2,132,154,6126,14.0
8,STYLES EARRINGS & STUDS,4.3,153,169,8033,9.0
9,FASHIONABLE EARRINGS & STUDS,4.2,89,923,3819,90.0


In [ ]:
url = "https://www.meesho.com/necklaces/pl/3lh"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_4= pd.DataFrame(all_data)
df_jewl_4["Discount %"] = round(100- (df_jewl_4["Current price"]/df_jewl_4['Previous price'])*100)
df_jewl_4

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,TRENDY NEW JEWELLERY SET,3.8,112,120,18166,7.0
1,SHIMMERING GRACEFUL WOMEN NECKLACES & CHAINS,4.3,110,129,13298,15.0
2,ELITE GRACEFUL WOMEN NECKLACES & CHAINS,4.0,127,136,5739,7.0
3,ALLURE FANCY WOMEN NECKLACES & CHAINS,4.1,85,85,15437,0.0
4,ELITE BEJEWELED WOMEN NECKLACES & CHAINS,4.1,128,139,2933,8.0
5,SIZZLING BEJEWELED WOMEN NECKLACES & CHAINS,4.1,289,289,0,0.0
6,SHIMMERING GLITTERING NECKLACE,4.0,208,208,0,0.0
7,ELITE CHARMING WOMEN NECKLACES & CHAINS,4.0,125,130,11735,4.0
8,ALLURE CHUNKY WOMEN NECKLACES & CHAINS,4.1,248,248,0,0.0
9,SIZZLING CHARMING WOMEN NECKLACES & CHAINS,3.9,270,270,0,0.0


In [ ]:
url = "https://www.meesho.com/rings-women/pl/3j8"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_5= pd.DataFrame(all_data)
df_jewl_5["Discount %"] = round(100- (df_jewl_5["Current price"]/df_jewl_5['Previous price'])*100)
df_jewl_5

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,SHIMMERING GRACEFUL RINGS,4.1,196,214,1816,8.0
1,ALLURE FANCY RINGS,4.1,195,213,10315,8.0
2,SHIMMERING CHUNKY RINGS,3.9,146,170,25247,14.0
3,ELITE FANCY RINGS,4.0,147,172,8305,15.0
4,TWINKLING GRACEFUL RINGS,4.3,137,142,3228,4.0
5,ALLURE CHUNKY RINGS,4.4,211,223,7695,5.0
6,TWINKLING GLITTERING RINGS,4.3,115,115,4513,0.0
7,SIZZLING GRACEFUL RINGS,4.2,167,182,1340,8.0
8,SHIMMERING GRACEFUL RINGS,4.1,177,193,5900,8.0
9,ALLURE BEJEWELED RINGS,4.2,138,141,1400,2.0


In [ ]:
url = "https://www.meesho.com/anklets-payal/pl/3je"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_jewl_6= pd.DataFrame(all_data)
df_jewl_6["Discount %"] = round(100- (df_jewl_6["Current price"]/df_jewl_6['Previous price'])*100)
df_jewl_6

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,SHIMMERING GLITTERING WOMEN ANKLETS & TOE RINGS,3.8,193,201,7140,4.0
1,ELITE GLITTERING WOMEN ANKLETS & TOE RINGS,3.9,216,216,2105,0.0
2,ALLURE FANCY WOMEN ANKLETS & TOE RINGS,4.0,87,969,12587,91.0
3,SIZZLING GRACEFUL WOMEN ANKLETS & TOE RINGS,4.0,218,229,1054,5.0
4,ELITE BEJEWELED WOMEN ANKLETS & TOE RINGS,3.8,144,161,3530,11.0
5,ELITE GLITTERING WOMEN ANKLETS & TOE RINGS,3.9,154,154,449,0.0
6,SIZZLING BEJEWELED WOMEN ANKLETS & TOE RINGS,3.7,123,123,1814,0.0
7,ELITE BEJEWELED WOMEN ANKLETS & TOE RINGS,3.8,112,122,5633,8.0
8,SHIMMERING CHARMING WOMEN ANKLETS & TOE RINGS,4.0,163,163,156,0.0
9,ALLURE GRACEFUL WOMEN ANKLETS & TOE RINGS,3.7,361,361,2211,0.0


In [ ]:
url = "https://www.meesho.com/bags-ladies/pl/3jo"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass= pd.DataFrame(all_data)
df_ass["Discount %"] = round(100- (df_jewl_6["Current price"]/df_jewl_6['Previous price'])*100)
df_jewl_6

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ELEGANT FASHIONABLE WOMEN HANDBAGS,4.0,212,234,31958,9.0
1,CLASSIC VERSATILE WOMEN SLINGBAGS,3.9,267,289,53,8.0
2,ELITE VERSATILE WOMEN SLINGBAGS,3.4,141,148,14,5.0
3,GORGEOUS ATTRACTIVE WOMEN SLINGBAGS,4.1,117,128,64451,9.0
4,TRENDY FANCY WOMEN SLINGBAGS,4.1,115,126,24387,9.0
5,ELEGANT FANCY WOMEN SLINGBAGS,3.0,200,200,10,0.0
6,TRENDY VERSATILE WOMEN SLINGBAGS,3.6,338,374,79,10.0
7,CLASSIC FANCY WOMEN SLINGBAGS,4.1,257,278,15223,8.0
8,RAVISHING VERSATILE WOMEN SLINGBAGS,3.2,297,322,24,8.0
9,RAVISHING FANCY WOMEN SLINGBAGS,3.8,274,274,0,0.0


In [ ]:
url = "https://www.meesho.com/women-watches/pl/3jx"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_1= pd.DataFrame(all_data)
df_ass_1["Discount %"] = round(100- (df_ass_1["Current price"]/df_ass_1['Previous price'])*100)
df_ass_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FEMALE WATCH,4.0,156,168,907,7.0
1,CLASSY WOMEN WATCHES,4.2,191,200,864,4.0
2,BOLUN UNIQUE WOMEN WATCHES,3.7,145,161,568,10.0
3,CLASSY WOMEN WATCHES,3.8,179,187,288,4.0
4,FABULOUS WOMEN ANALOG WATCHES,3.9,130,138,823,6.0
5,CLASSIC WOMEN WATCHES,3.9,182,182,660,0.0
6,UNIQUE WOMEN WATCHES,3.9,147,159,764,8.0
7,TRENDY WOMEN WATCHES,4.0,187,202,896,7.0
8,TRENDY WOMEN WATCHES,4.1,177,184,704,4.0
9,ATTRACTIVE WOMEN ANALOG WATCHES,3.8,136,141,192,4.0


In [ ]:
url = "https://www.meesho.com/hair-accessories-women/pl/3jl"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_2= pd.DataFrame(all_data)
df_ass_2["Discount %"] = round(100- (df_ass_2["Current price"]/df_ass_2['Previous price'])*100)
df_ass_2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ELITE FANCY WOMEN HAIR ACCESSORIES,4.4,148,160,1931,8.0
1,PRINCESS FANCY JEWELLERY SETS,,998,998,0,0.0
2,ALLURE COLORFUL WOMEN HAIR ACCESSORIES,3.7,142,142,833,0.0
3,ELITE GLITTERING WOMEN HAIR ACCESSORIES,4.0,133,142,470,6.0
4,ALLURE FUSION WOMEN HAIR ACCESSORIES,4.0,187,206,289,9.0
5,DIVA CHUNKY WOMEN HAIR ACCESSORIES,4.0,173,191,1732,9.0
6,TRENDY HAIR ACCESSORIES,4.2,176,194,804,9.0
7,DIVA FUSION WOMEN HAIR ACCESSORIES,3.8,117,125,310,6.0
8,FEMININE ELEGANT WOMEN HAIR ACCESSORIES,4.2,134,137,2066,2.0
9,SHIMMERING FANCY WOMEN HAIR ACCESSORIES,4.2,91,104,1855,12.0


In [ ]:
url = "https://www.meesho.com/sunglasses-women/pl/3kd"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_3= pd.DataFrame(all_data)
df_ass_3["Discount %"] = round(100- (df_ass_3["Current price"]/df_ass_3['Previous price'])*100)
df_ass_3

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,WOMEN OVAL SUNGLASSES,4.1,138,143,1771,3.0
1,WOMEN SQUARE SUNGLASSES,4.3,402,402,25,0.0
2,WOMEN ROUND SUNGLASSES,4.0,146,158,11635,8.0
3,WOMEN RECTANGULAR SUNGLASSES,4.2,169,177,1352,5.0
4,WOMEN WAYFARER SUNGLASSES,3.9,118,126,2883,6.0
5,STYLES TRENDY WOMEN SPECTACLE FRAMES,3.8,203,203,0,0.0
6,WOMEN SQUARE SUNGLASSES,4.2,130,140,1827,7.0
7,WOMEN ROUND SUNGLASSES,3.6,131,136,2096,4.0
8,WOMEN OVAL SUNGLASSES,4.2,119,127,6645,6.0
9,WOMEN OVAL SUNGLASSES,4.0,119,143,1318,17.0


In [ ]:
url = "https://www.meesho.com/socks-women/pl/3km"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_4= pd.DataFrame(all_data)
df_ass_4["Discount %"] = round(100- (df_ass_4["Current price"]/df_ass_4['Previous price'])*100)
df_ass_4

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FANCY TRENDY WOMEN SOCKS,4.2,128,128,2254,0.0
1,STYLES UNIQUE WOMEN SOCKS,4.3,123,134,13700,8.0
2,CASUAL LATEST WOMEN SOCKS,4.4,206,211,848,2.0
3,CASUAL UNIQUE WOMEN SOCKS,4.2,131,143,4077,8.0
4,CASUAL MODERN WOMEN SOCKS,4.4,189,209,6835,10.0
5,CASUAL UNIQUE WOMEN SOCKS,4.0,124,127,2235,2.0
6,WINTER WOOL SOCKS WOMENS,4.2,128,137,9085,7.0
7,FASHIONABLE UNIQUE WOMEN SOCKS,4.2,124,135,3619,8.0
8,CASUAL TRENDY WOMEN SOCKS,3.9,163,163,0,0.0
9,CASUAL LATEST WOMEN SOCKS,3.9,112,122,3359,8.0


In [ ]:
url = "https://www.meesho.com/flats-women/pl/3o1"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_5= pd.DataFrame(all_data)
df_ass_5["Discount %"] = round(100- (df_ass_5["Current price"]/df_ass_5['Previous price'])*100)
df_ass_5

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,FANCY WOMEN FLATS,3.8,167,173,4743,3.0
1,GORGEOUS WOMEN FLATS,3.1,291,291,14,0.0
2,TRENDY WOMEN FLATS,4.2,326,383,17336,15.0
3,ELITE WOMEN FLATS,3.9,207,225,3225,8.0
4,ATTRACTIVE WOMEN FLATS,4.1,325,382,3787,15.0
5,FANCY WOMEN FLATS,4.0,162,170,5890,5.0
6,MODERN WOMEN FLATS,3.8,163,173,1808,6.0
7,SHOESTAIL FLATS FOR WOMEN,4.1,288,999,1322,71.0
8,COLORFUL WOMEN FLATS,4.1,175,205,3977,15.0
9,ELITE WOMEN FLATS,4.1,235,286,1800,18.0


In [ ]:
url = "https://www.meesho.com/bellies/pl/3n8"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_6= pd.DataFrame(all_data)
df_ass_6["Discount %"] = round(100- (df_ass_6["Current price"]/df_ass_6['Previous price'])*100)
df_ass_6

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,MODERN BELLIES,4.0,197,225,207,12.0
1,ESSENTIAL BELLIES,3.9,305,305,0,0.0
2,UNIQUE BELLIES,4.1,241,266,1516,9.0
3,WONDERFUL BELLIES,3.9,140,164,1945,15.0
4,GRACEFUL BELLIES,3.9,140,164,1784,15.0
5,FASHIONABLE BELLIES,3.9,284,284,0,0.0
6,STYLES BELLIES,3.9,257,257,0,0.0
7,FANCY BELLIES,4.1,263,290,4049,9.0
8,WONDERFUL BELLIES,3.9,236,280,1191,16.0
9,FANCY BELLIES,3.7,173,197,1753,12.0


In [ ]:
url = "https://www.meesho.com/jutti/pl/3lx"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_ass_7= pd.DataFrame(all_data)
df_ass_7["Discount %"] = round(100- (df_ass_7["Current price"]/df_ass_7['Previous price'])*100)
df_ass_7

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,LATEST WOMEN JUTTIS & MOJARIS,3.9,162,187,16325,13.0
1,CLASSY WOMEN JUTTIS & MOJARIS,4.0,272,272,0,0.0
2,ELITE WOMEN JUTTIS & MOJARIS,4.0,140,155,2842,10.0
3,ATTRACTIVE WOMEN JUTTIS & MOJARIS,4.1,208,221,11465,6.0
4,VOGUISH WOMEN JUTTIS & MOJARIS,4.1,167,186,4835,10.0
5,GORGEOUS WOMEN JUTTIS & MOJARIS,4.0,149,166,4381,10.0
6,MODERN WOMEN JUTTIS & MOJARIS,3.8,183,195,4044,6.0
7,ALLURING WOMEN JUTTIS & MOJARIS,4.0,142,158,398,10.0
8,+21 MOREATTRACTIVE FANCY WOMEN'S ETHINC FLATS ...,3.8,163,176,6009,7.0
9,FANCY WOMEN JUTTIS & MOJARIS,4.3,350,402,1127,13.0


##Electronic

In [ ]:
url = "https://www.meesho.com/mobiles-and-accessories/pl/3tk"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_elec= pd.DataFrame(all_data)
df_elec["Discount %"] = round(100- (df_elec["Current price"]/df_elec['Previous price'])*100)
df_elec

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,CABLES,3.8,279,279,0,0.0
1,MOBILE CHARGERS,3.8,171,204,7567,16.0
2,MOBILE CHARGERS,3.8,180,205,3914,12.0
3,MOBILE CHARGERS,3.7,161,184,1802,12.0
4,MOBILE CHARGERS,3.8,174,199,3837,13.0
5,CABLES,4.0,109,109,3177,0.0
6,MOBILE CHARGERS,3.9,156,186,1930,16.0
7,CABLES,4.0,167,180,3960,7.0
8,MOBILE CHARGERS,3.9,151,180,4566,16.0
9,MI REDMI 8 CASES & COVERS,4.2,135,154,2666,12.0


In [ ]:
url = "https://www.meesho.com/mobile-cases-covers/pl/3q3"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_elec_1= pd.DataFrame(all_data)
df_elec_1["Discount %"] = round(100- (df_elec_1["Current price"]/df_elec_1['Previous price'])*100)
df_elec_1

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,"+16 MOREVIVO Y16, VIVO Y56 5G CASES & COVERS",3.9,131,150,537,13.0
1,MI REDMI 8 CASES & COVERS,4.2,135,154,2666,12.0
2,POCO M3 CASES & COVERS,3.9,139,157,658,11.0
3,MI REDMI 9 CASES & COVERS,4.0,146,171,246,15.0
4,REALME 6 CASES & COVERS,4.0,137,149,987,8.0
5,"TECNO SPARK 6 GO,TECNO SPARK GO 2020 CASES & C...",3.9,130,138,16,6.0
6,MI REDMI 8A DUAL CASES & COVERS,4.2,146,171,1320,15.0
7,SAMSUNG GALAXY M31 CASES & COVERS,4.1,140,164,453,15.0
8,MI REDMI 7A CASES & COVERS,4.1,151,172,830,12.0
9,MI REDMI 9A CASES & COVERS,4.0,116,142,579,18.0


In [ ]:
url = "https://www.meesho.com/appliances/pl/3to"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_elec_2= pd.DataFrame(all_data)
df_elec_2["Discount %"] = round(100- (df_elec_2["Current price"]/df_elec_2['Previous price'])*100)
df_elec_2

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,EXTENSION CHORDS,3.6,197,208,1680,5.0
1,AMAZING WASHING MACHINES & DRYERS,4.0,442,496,816,11.0
2,USEFUL EXTENSION BOARDS,3.7,133,156,111687,15.0
3,FANCY EXTENSION CORD MANAGEMENT,3.6,131,155,6110,15.0
4,LATEST IRONS,3.7,304,304,6853,0.0
5,FANCY FOOD PROCESSOR SET,4.0,149,161,502,7.0
6,PORTRONICS POWER PLATE 10 1500W POWER CONVERTE...,3.9,373,699,60,47.0
7,ELEGANT IRONS,4.0,350,367,12833,5.0
8,TRENDY EXTENSION CHORDS,3.7,289,303,10,5.0
9,TRENDY EXTENSION CHORDS,4.2,288,302,6,5.0


In [ ]:
url = "https://www.meesho.com/grooming-kits-men/pl/3kn"
page_content = r.get(url).content
soup = BeautifulSoup(page_content, 'html.parser')


all_class = []

for i in soup.find_all('div', {"class": "sc-gswNZR sc-hLBbgP JoMRc etQEgJ products", "class": True}):
  if "sc-dkrFOg" in i["class"]:
    for j in i["class"]:
      if j == "cokuZA":
        all_class.append(tuple(i["class"]))

all_class = list((set(all_class)))

import re

all_rating = []
all_product_name = []
original_price = []
Dis_price = []
Review =[]

for i in all_class:
  html_phar = " ".join(i)

  for index,j in enumerate(soup.find_all("div", {"class":html_phar})):
    product_info = j.text.strip().upper()
    #print(product_info)

    # Rating
    rating = product_info[product_info.find('DELIVERY'):product_info.find('DELIVERY')+len("DELIVERY")+3].replace("DELIVERY","")
    all_rating.append(rating)
    #print(rating)

    #Product Name
    product_name = product_info.split("₹")[0]
    #print(product_name)

    ptr1 = r'\+\d MORE'
    res = re.findall(ptr1, product_name)
    if len(res) >0:
      temp_pro_name = product_name.replace(res[0],"")
      #print(temp_pro_name)

      ptr3 = r'\d{2}H : \d{2}M : \d{2}S'
      res1 = re.findall(ptr3,temp_pro_name)
      if len(res1) >0:
        all_product_name.append(temp_pro_name.replace(res1[0],""))
      else:
        all_product_name.append(temp_pro_name)
    else:

      ptr2 = r'\d{2}H : \d{2}M : \d{2}S'
      res = re.findall(ptr2,product_name)
      #print(res)
      if len(res) >0:
        all_product_name.append(product_name.replace(res[0],""))
      else:
        all_product_name.append(product_name)

    #Review
    rew = re.findall(r'DELIVERY\d\.\d(\d{1,}) REVIEWS', product_info)
    if len(rew) > 0:
      Review.append(int(rew[0]))
    else:
      Review.append(0)


    #Price
    #print(product_info)
    ptr4 = r'₹\d{1,}%?'
    res2= re.findall(ptr4,product_info)

    if len(res2)==1:
      current_price = res2[0]
      cp_1 = current_price.strip().replace("₹","")
      Dis_price.append(int(cp_1))
      original_price.append(int(cp_1))

    elif len(res2) == 2:
      current_price,previous_price = res2
      pp= previous_price[:4].replace("₹","").strip()
      cp = current_price[:4].replace("₹","").strip()

      original_price.append(int(pp))
      Dis_price.append(int(cp))

all_data = {"Product Name": all_product_name, "Ratings": all_rating, "Current price": Dis_price, "Previous price" : original_price, "No of Review" : Review}

df_elec_3= pd.DataFrame(all_data)
df_elec_3["Discount %"] = round(100- (df_elec_3["Current price"]/df_elec_3['Previous price'])*100)
df_elec_3

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,ADVANCED PROCTECTIVE BEARD OIL & WAX,3.9,184,193,571,5.0
1,USTRAA BEARD GROWTH OIL - 35ML,3.9,254,399,5492,36.0
2,PROFFESIONAL INTENSIVE BEARD OIL & WAX,4.1,134,139,1644,4.0
3,"ARSH ORGANICS BEARD AND HAIR GROWTH OIL, 30 ML...",4.1,130,137,12427,5.0
4,ADVANCED URBAN MEN BEARD HAIR GROWTH OIL- BEST...,4.1,119,133,4680,11.0
5,"USTRAA BEARD OIL (BEARD GROWTH OIL ADVANCED, B...",3.9,408,699,516,42.0
6,MANCODE AYURVEDIC BEARD GROWTH OIL FOR MEN FAS...,4.1,147,499,253,71.0
7,NEW RAZORS & CARTRIDGES,3.8,107,118,1850,9.0
8,TRENDY BEARD OIL,3.6,126,126,0,0.0
9,TRENDY BEARD OIL,3.6,126,126,0,0.0


In [ ]:
full_data_3= pd.concat([df_jewl_1,df_jewl,df_jewl_2,df_jewl_3,df_jewl_4,df_jewl_5,df_jewl_6,df_ass,df_ass_1,df_ass_2,df_ass_3,df_ass_4,df_ass_5,df_ass_6,df_ass_7,df_elec,df_elec_1,df_elec_2,df_elec_3],ignore_index=True)
full_data_3

,Product Name,Ratings,Current price,Previous price,No of Review,Discount %
0,TRENDY EARRINGS & STUDS,4.0,160,168,6215,5.0
1,TRENDY OTHERS (EARRINGS DEP),3.9,159,159,36,0.0
2,LATEST EARRINGS & STUDS,4.2,126,131,5196,4.0
3,GRACEFUL EARRINGS & STUDS,4.2,155,181,54286,14.0
4,UNIQUE EARRINGS & STUDS,4.2,141,160,5082,12.0
...,...,...,...,...,...,...
374,TRENDY BEARD OIL,3.6,144,144,0,0.0
375,MANCODE BEARD GROWTH OIL FOR MEN 50ML,3.9,176,499,572,65.0
376,CASUAL RAZORS & CARTRIDGES,4.1,102,114,1054,11.0
377,PROFFESIONAL INTENSIVE BEARD OIL & WAX,3.8,118,132,240,11.0


In [ ]:
full_data_3.to_csv("Jewellary_elec data.csv")